In [5]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('./model/all-MiniLM-EV34-L6-v2')
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#bad model - model = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb')
#model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

#Compute embedding for both lists
#sentences = ["POST STERNOTOMY SCAR", "Breast biopsy"]
#embedding_1= model.encode(sentences[0], convert_to_tensor=True)
#embedding_2 = model.encode(sentences[1], convert_to_tensor=True)
#util.pytorch_cos_sim(embedding_1, embedding_2)
## tensor([[0.6003]])

In [6]:
#Passed the variable sent into the model and created the embeddings
df = pd.read_csv(r".\data\csv\meddra26_1_llt.csv")
embeddings = model.encode(df['llt_name'], convert_to_tensor=True)
df['LLT_Embeddings'] = embeddings.tolist()

df.to_pickle(r".\data\csv\meddra26_1_llt_embeddings_EV34.pkl")  # where to save it, usually as a .pkl

In [8]:
#Passed the variable sent into the model and created the embeddings
df1 = pd.read_csv(r".\data\csv\plate026.csv")
q_embeddings = model.encode(df1['aeterm'], convert_to_tensor=True)
df1['q_Embeddings'] = q_embeddings.tolist()

df1.to_pickle(r".\data\csv\plate026_embeddings_EV34.pkl")  # where to save it, usually as a .pkl

In [ ]:

df_combined=df1.merge(df, how='cross')
df_combined


In [ ]:
df_combined['Cos_Sim']=df_combined.apply(lambda x: util.pytorch_cos_sim(x['LLT_Embeddings'], x['q_Embeddings']).data[0,0].numpy(), axis=1)
df_combined = df_combined.sort_values('Cos_Sim',ascending = False).groupby('aeterm').head(1)
df_combined


aeterm            llt_name_coded  \
1043917     PERIPHERAL NEUROPATHY     Peripheral neuropathy   
253236     INTERMITTENT HEADACHES     Intermittent headache   
880837        HIGH BLOOD PRESSURE       Blood pressure high   
509916         SEASONAL ALLERGIES          Seasonal allergy   
611140            POST-NASAL DRIP            Postnasal drip   
1178102     CHRONIC LOW BACK PAIN             Low back pain   
411104    ENVIRONMENTAL ALLERGIES     Environmental allergy   
316989            LOWER BACK PAIN             Low back pain   
706835        LEFT ANKLE FRACTURE            Ankle fracture   
216669            DIABETES TYPE 2  Type 2 diabetes mellitus   
1099223    RIGHT KNEE ARTHROSCOPY        Arthroscopy R knee   
1255631       RIGHT SHOULDER PAIN             Shoulder pain   
1428129     RIGHT HIP REPLACEMENT           Hip replacement   
1552972  TORN MENISCUS RIGHT KNEE     Meniscus tear of knee   
1346369      INTERMITTENT FATIGUE                   Fatigue   
959120    ELEVATED BLOOD PRESSURE  Increased blood pressure   
16851                 ACID REFLUX  Acid reflux (esophageal)   
835526              POOR APPETITE        Decreased appetite   
645999     WORSENING HYPERTENSION     Hypertension worsened   
97969             CODEINE ALLERGY   Specific allergy (drug)   

                                              q_Embeddings  \
1043917  [-0.023981483653187752, -0.1295260488986969, 0...   
253236   [-0.07385068386793137, -0.02499891258776188, 0...   
880837   [-0.08094657957553864, 0.08691059798002243, -0...   
509916   [0.004224624950438738, -0.02293059229850769, 0...   
611140   [0.015070932917296886, 0.02133166790008545, 0....   
1178102  [0.06704182177782059, -0.029366787523031235, -...   
411104   [0.02507692389190197, 0.006804064381867647, 0....   
316989   [0.008245270699262619, -0.02171381190419197, 7...   
706835   [-0.10110331326723099, 0.0043624709360301495, ...   
216669   [0.016450541093945503, 0.022364230826497078, 0...   
1099223  [-0.09925615787506104, 0.07034490257501602, -0...   
1255631  [-0.03390416502952576, -0.0049247643910348415,...   
1428129  [-0.06909634917974472, 0.09367772191762924, 0....   
1552972  [-0.038358572870492935, 0.05613192915916443, -...   
1346369  [-0.03819163888692856, -0.05205097422003746, 0...   
959120   [-0.07816831022500992, 0.0730028823018074, -0....   
16851    [0.030315151438117027, -0.06729761511087418, -...   
835526   [0.02993662841618061, 0.01969110779464245, 0.0...   
645999   [-0.014824125915765762, 0.08546262979507446, 0...   
97969    [-0.007714566774666309, -0.09199722856283188, ...   

                          llt_name  llt_code  vernum  \
1043917      Peripheral neuropathy  10080342    26.1   
253236       Intermittent headache  10059296    26.1   
880837         Blood pressure high  10005747    26.1   
509916            Seasonal allergy  10048908    26.1   
611140     Chronic post nasal drip  10066649    26.1   
1178102              Low back pain  10024891    26.1   
411104       Environmental allergy  10048696    26.1   
316989               Low back pain  10024891    26.1   
706835              Ankle fracture  10002544    26.1   
216669    Type 2 diabetes mellitus  10067585    26.1   
1099223           Knee arthroscopy  10077420    26.1   
1255631              Shoulder pain  10040617    26.1   
1428129            Hip replacement  10020102    26.1   
1552972      Meniscus tear of knee  10027291    26.1   
1346369            Chronic fatigue  10066564    26.1   
959120         Blood pressure high  10005747    26.1   
16851    Acid reflux (oesophageal)  10000479    26.1   
835526               Appetite lost  10003028    26.1   
645999       Hypertension worsened  10020799    26.1   
97969                      Allergy  10001738    26.1   

                                            LLT_Embeddings     Cos_Sim  
1043917  [-0.023981483653187752, -0.1295260488986969, 0...         1.0  
253236   [-0.06853340566158295, -0.012225408107042313, ...  0.97960734  
8808

In [ ]:
acc = (df_combined.llt_name==df_combined.llt_name_coded).mean()
acc

0.7

In [ ]:
#df1.drop(columns="Unnamed: 0", inplace=True)
#df1.drop(columns="Unnamed: 0.1", inplace=True)
#df1

aeterm  \
0                                            ACID REFLUX   
1                                        CODEINE ALLERGY   
2                                        DIABETES TYPE 2   
3                                 INTERMITTENT HEADACHES   
4                                        LOWER BACK PAIN   
...                                                  ...   
93317  SUBTLE JOINT PAIN EXPERIENCED IN THE ELBOWS BI...   
93318             SULFUR/SULFONAMIDE ANTIBIOTICS ALLERGY   
93319                             THROMBOLYSIS PROCEDURE   
93320                    UNILATERAL SALPINGOOOPHORECTOMY   
93321                            UNINARY TRACT INFECTION   

                         llt_name_coded  \
0              Acid reflux (esophageal)   
1               Specific allergy (drug)   
2              Type 2 diabetes mellitus   
3                 Intermittent headache   
4                         Low back pain   
...                                 ...   
93317                     Pain in elbow   
93318               Sulfonamide allergy   
93319                      Thrombolysis   
93320  Salpingo-oophorectomy unilateral   
93321           Urinary tract infection   

                                            q_Embeddings  
0      [0.03031516633927822, -0.06729763746261597, -0...  
1      [-0.007714566774666309, -0.09199722856283188, ...  
2      [0.016450542956590652, 0.022364258766174316, 0...  
3      [-0.07385068386793137, -0.02499891258776188, 0...  
4      [0.008245265111327171, -0.02171381376683712, 7...  
...                                                  ...  
93317  [0.0035349878016859293, -0.008019516244530678,...  
93318  [-0.005005732644349337, -0.05093630775809288, ...  
93319  [-0.08474110811948776, 0.03592824935913086, -0...  
93320  [0.021358858793973923, 0.026225747540593147, 0...  
93321  [-0.026780012995004654, -0.09017617255449295, ...  

[93322 rows x 3 columns]

In [11]:
#Testing
from sentence_transformers import SentenceTransformer, util
import pandas as pd

df = pd.read_pickle(r".\data\csv\meddra26_1_llt_embeddings_EV34.pkl")
df1 = pd.read_pickle(r".\data\csv\plate026_embeddings_EV34.pkl")
df_combined=df1[:100].merge(df, how='cross')
#df_combined=df1[df1['aeterm']=="LEFT MENISCAL SURGERY"].merge(df, how='cross')

df_combined['Cos_Sim']=df_combined.apply(lambda x: util.pytorch_cos_sim(x['LLT_Embeddings'], x['q_Embeddings']).data[0,0].numpy(), axis=1)
df_combined = df_combined.sort_values('Cos_Sim',ascending = False).groupby('aeterm').head(1)
acc = (df_combined.llt_name==df_combined.llt_name_coded).mean()
acc
df_combined

#Base - 66%
#EV2-epoch=2 - 68%
#EV6 (version 21)-epoch=6 - 72%
#EV34 (version 26.1)-epoch=6 - 76%


aeterm  \
7084787                                           VOMITING   
6286182                         LYMPHOCYTE COUNT DECREASED   
5421211                                  ALLERGIC REACTION   
6392979                                       MALNUTRITION   
4091444                                      MONONUCLEOSIS   
...                                                    ...   
3855491                                        FAR SIGHTED   
6575872                                           PAIN-ABD   
7252253                                      DECREASED ANC   
6925006                           TENDERNESS IN LOWER BACK   
5247741  TENDERNESS IN THE UMBILICAL AREA AND AT SITE O...   

                              llt_name_coded  \
7084787                             Vomiting   
6286182           Lymphocyte count decreased   
5421211                Allergic reaction NOS   
6392979                         Malnutrition   
4091444               Mononucleosis syndrome   
...                                      ...   
3855491                           Farsighted   
6575872                       Abdominal pain   
7252253  Absolute neutrophil count decreased   
6925006                        Low back pain   
5247741                   Periumbilical pain   

                                              q_Embeddings  \
7084787  [0.08193761855363846, -0.010552980937063694, -...   
6286182  [0.05710356682538986, -0.024302957579493523, 0...   
5421211  [-0.042998023331165314, 0.017401859164237976, ...   
6392979  [0.0741846114397049, 0.11459909379482269, -0.0...   
4091444  [-0.006896820850670338, -0.016407987102866173,...   
...                                                    ...   
3855491  [0.07249051332473755, 0.00308405258692801, -0....   
6575872  [0.036881886422634125, -0.007143881171941757, ...   
7252253  [-0.036951594054698944, -0.02674456126987934, ...   
6925006  [0.02082531899213791, -0.05685104429721832, -0...   
5247741  [0.03075689822435379, 0.006559754256159067, -0...   

                            llt_name  llt_code  vernum  \
7084787                     Vomiting  10047700    26.1   
6286182   Lymphocyte count decreased  10025256    26.1   
5421211            Allergic reaction  10001718    26.1   
6392979                 Malnutrition  10061273    26.1   
4091444                Mononucleosis  10079260    26.1   
...                              ...       ...     ...   
3855491                   Farsighted  10057906    26.1   
6575872               Pain abdominal  10033374    26.1   
7252253             c-ANCA decreased  10060180    26.1   
6925006                   Tenderness  10043224    26.1   
5247741  Application site tenderness  10058714    26.1   

                                            LLT_Embeddings     Cos_Sim  
7084787  [0.08193761855363846, -0.010552980937063694, -...   1.0000002  
6286182  [0.05710356682538986, -0.024302957579493523, 0...   1.0000002  
5421211  [-0.04299800843000412, 0.017401887103915215, -...   1.0000002  
6392979  [0.0741846114397049, 0.11459909379482269, -0.0...   1.0000001  
4091444  [-0.006896820850670338, -0.016407987102866173,...   1.0000001  
...                                                    ...         ...  
3855491  [0.059170182794332504, 0.05162133648991585, -0...   0.7770457  
6575872  [0.08463002741336823, 0.021026501432061195, -0...  0.75718397  
7252253  [0.008832189254462719, 0.02741899900138378, -0...  0.73239887  
6925006  [-0.005652198102325201, -0.025643039494752884,...   0.7124206  
5247741  [0.014192360453307629, -0.036859508603811264, ...   0.6360022  

[100 rows x 8 columns]

In [ ]:
df_combined


aeterm      llt_name_coded  \
65591  LEFT MEDIAL MENISCUS REPAIR  Meniscus operation   

                                            q_Embeddings         llt_name  \
65591  [1.227443317475263e-05, -0.012787006795406342,...  Meniscus injury   

       llt_code  vernum                                     LLT_Embeddings  \
65591  10072970    26.1  [0.02747674100100994, 0.010407702066004276, -0...   

         Cos_Sim  
65591  0.7642219